In [2]:

import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from PIL import Image
import traceback

print("--- Initializing MoodMate ---")

# --- Step 2: Define the MusicRecommender Class ---
class MusicRecommender:
    def __init__(self, data_path):
        self.emotion_to_music_map = {
            'happy': 'upbeat happy energetic dance pop joy', 'sad': 'sad mellow slow acoustic blues classical instrumental',
            'angry': 'angry rock metal intense heavy punk industrial', 'fear': 'ambient experimental instrumental calm classical soothing',
            'surprise': 'electronic pop dance energetic new wave synthpop', 'neutral': 'lounge chill instrumental ambient pop easy listening',
            'disgust': 'industrial metal experimental rock'
        }
        self.df = self._load_data(data_path)
        if not self.df.empty:
            self.tfidf_vectorizer = TfidfVectorizer(stop_words='english')
            self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.df['tags'])
            print(" Music Recommender initialized.")

    def _load_data(self, data_path):
        if not os.path.exists(data_path):
            print(f" ERROR: Music data not found at '{data_path}'. Please check your project structure.")
            return pd.DataFrame()
        return pd.read_csv(data_path)

    def recommend_songs(self, emotion, num_recommendations=10):
        if self.df.empty: return pd.DataFrame()
        query_tags = self.emotion_to_music_map.get(emotion.lower())
        if not query_tags: return pd.DataFrame()
        query_vector = self.tfidf_vectorizer.transform([query_tags])
        cosine_similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        top_song_indices = cosine_similarities.argsort()[:-num_recommendations-1:-1]
        return self.df.iloc[top_song_indices][['artist_name', 'title']]

# --- Step 3: Configuration and Model Loading ---

MODEL_PATH = 'models/cnn_emotion_model.h5' # Or: 'models/vgg16_emotion_model_50_epochs.h5'
MUSIC_DATA_PATH = 'data/music_processed/processed_music_tags.csv'
EMOTION_MAP = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}

try:
    emotion_model = tf.keras.models.load_model(MODEL_PATH)
    print(f" Emotion model loaded successfully from '{MODEL_PATH}'.")
except Exception as e:
    print(f" FATAL ERROR: Could not load the emotion model from '{MODEL_PATH}'.")
    print("Please ensure your notebook is in the main project folder and the path is correct.")
    emotion_model = None
    
music_recommender = MusicRecommender(MUSIC_DATA_PATH)

# --- Step 4: Define the Prediction Function ---
def predict_emotion_from_image_bytes(image_bytes, model):
    if model is None:
        print(" ERROR: Emotion model is not loaded. Cannot predict.")
        return None
    try:
        if isinstance(image_bytes, str): image_bytes = image_bytes.encode('utf-8')
        nparr = np.frombuffer(image_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(" ERROR: OpenCV could not decode the image.")
            return None
        if model.input_shape[-1] == 3: img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img_resized = cv2.resize(img, (48, 48))
        img_normalized = img_resized / 255.0
        img_expanded = np.expand_dims(img_normalized, axis=0)
        if len(img_expanded.shape) < 4 and model.input_shape[-1] == 1: img_expanded = np.expand_dims(img_expanded, axis=-1)
        prediction = model.predict(img_expanded, verbose=0)
        return EMOTION_MAP[np.argmax(prediction)]
    except Exception as e:
        print(f" An unexpected error occurred during prediction: {e}")
        traceback.print_exc()
        return None

# --- Step 5: Create and Display the Interactive UI ---
uploader = widgets.FileUpload(accept='image/*', description='Upload Image', button_style='primary')
output_area = widgets.Output()

def on_file_upload(change):
    with output_area:
        clear_output(wait=True)
        uploaded_file = change['new']
        if not uploaded_file: return
        file_info = uploaded_file[0]
        image_bytes = file_info['content']
        
        print("--- Analyzing New Image ---")
        display(Image.open(io.BytesIO(image_bytes)))
        
        detected_emotion = predict_emotion_from_image_bytes(image_bytes, emotion_model)
        
        if detected_emotion:
            print(f"\n Detected Emotion: {detected_emotion.upper()}")
            playlist = music_recommender.recommend_songs(detected_emotion)
            if not playlist.empty:
                print("\n--- 🎶 Here is your personalized playlist ---")
                display(playlist)

                # --- NEW ANALYSIS SECTION ---
                print("\n--- How This Playlist Was Chosen ---")
                emotion_keywords = music_recommender.emotion_to_music_map.get(detected_emotion, "")
                print(f"1. Emotion Detected: The AI model analyzed the image and identified the emotion as '{detected_emotion}'.")
                print(f"2. Keyword Mapping: This emotion was then translated into a set of musical keywords: '{emotion_keywords}'.")
                print("3. Song Matching: Finally, the system searched a library of songs to find the ones whose tags most closely matched these keywords.")

        else:
            print("\n Could not detect an emotion. Please check the error messages above.")

uploader.observe(on_file_upload, names='value')

print("\n--- MoodMate Interactive UI ---")
print("Upload an image of a face to get started.")
display(uploader, output_area)

2025-10-15 23:03:54.519 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 23:03:54.525 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-15 23:03:54.530 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 23:03:58.447 
  command:

    streamlit run C:\Users\SURYA PRATAP DASH\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-15 23:03:58.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 23:03:58.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 23:03:58.456 Thread 'MainThread': missing ScriptRunContex